### Настройки

In [1]:
# импорт библиотек
import pandas as pd
import psycopg2

import warnings
warnings.filterwarnings("ignore")

In [2]:
# данные коннекта с postregSQL
# при размещении в github удалите эту ячейку

DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

Cоединение с заданными параметрами

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

Задание 3. Знакомство с данными. Предварительный анализ данных

In [4]:
# определяем количество вакансий в базе
v_count_query = 'select count(*) from vacancies'
v_count = pd.read_sql_query(v_count_query,connection)
print('Число вакансий в базе данных:', v_count.iloc[0][0])

#определяем число работодателей в базе
e_count_query = 'select count(*) from employers'
e_count = pd.read_sql_query(e_count_query,connection)
print('Число работодателей в базе данных:', e_count.iloc[0][0])

#определяем число регионов в базе
a_count_query = 'select count(*) from areas'
a_count = pd.read_sql_query(a_count_query,connection)
print('Число регионов в базе данных:', a_count.iloc[0][0])

#определяем число сфер деятельности в базе
i_count_query = 'select count(*) from industries'
i_count = pd.read_sql_query(i_count_query,connection)
print('Число сфер деятельности в базе данных:', i_count.iloc[0][0])



Число вакансий в базе данных: 49197
Число работодателей в базе данных: 23501
Число регионов в базе данных: 1362
Число сфер деятельности в базе данных: 294


Задание 4. Детальное исследование вакансий

In [5]:
# Распределение числа вакансий по регионам
query = '''select a.name area, count(v.id) cnt 
        from vacancies v left join areas a on v.area_id=a.id 
        group by a.name order by cnt desc'''
vacancies_by_area = pd.read_sql_query(query, connection)
print(vacancies_by_area.head(30))

               area   cnt
0            Москва  5333
1   Санкт-Петербург  2851
2             Минск  2112
3       Новосибирск  2006
4            Алматы  1892
5      Екатеринбург  1698
6   Нижний Новгород  1670
7            Казань  1415
8         Краснодар  1301
9            Самара  1144
10   Ростов-на-Дону  1131
11          Воронеж  1063
12          Ташкент   851
13       Красноярск   847
14       Нур-Султан   840
15        Челябинск   786
16            Пермь   771
17              Уфа   767
18            Томск   692
19             Омск   617
20          Саратов   579
21           Тюмень   547
22      Владивосток   539
23          Тбилиси   474
24           Ижевск   466
25        Волгоград   456
26          Иркутск   451
27        Ярославль   391
28          Барнаул   374
29           Рязань   346


In [6]:
# Число вакансий с хотя бы одним непустым полем зарплаты
query = '''select count(*) cnt from vacancies 
        where (salary_from is not null) or (salary_to is not null)'''
salary_not_null_cnt = pd.read_sql_query(query, connection)
print('Число строк с хотя бы одним непустым полем зарплаты:', salary_not_null_cnt.iloc[0][0])

# Среднее значение для нижней и верхней границ зарплатной вилки
query = '''select avg(salary_from) avg_from, avg(salary_to) avg_to from vacancies'''
salary_mean = pd.read_sql_query(query, connection)
print('Средние значения границ зарплатной вилки:', round(salary_mean['avg_from'].iloc[0]), round(salary_mean['avg_to'].iloc[0]))

Число строк с хотя бы одним непустым полем зарплаты: 24073
Средние значения границ зарплатной вилки: 71065 110537


In [7]:
# Распределение вакансий по совокупности график работы - занятость
query = '''select schedule, employment, count(id) number from vacancies 
group by schedule, employment order by number desc'''
sched_vac = pd.read_sql_query(query, connection)
print(sched_vac)


            schedule           employment  number
0        Полный день     Полная занятость   35367
1   Удаленная работа     Полная занятость    7802
2      Гибкий график     Полная занятость    1593
3   Удаленная работа  Частичная занятость    1312
4     Сменный график     Полная занятость     940
5        Полный день           Стажировка     569
6     Вахтовый метод     Полная занятость     367
7        Полный день  Частичная занятость     347
8      Гибкий график  Частичная занятость     312
9        Полный день     Проектная работа     141
10  Удаленная работа     Проектная работа     133
11     Гибкий график           Стажировка     116
12    Сменный график  Частичная занятость     101
13  Удаленная работа           Стажировка      64
14     Гибкий график     Проектная работа      18
15    Сменный график           Стажировка      12
16    Вахтовый метод     Проектная работа       2
17    Сменный график     Проектная работа       1


In [8]:
# Распределение вакансий в зависимости от опыта работы
query = '''select experience, count(id) number from vacancies
        group by experience order by number desc'''

experience = pd.read_sql_query(query, connection)
print(experience)

           experience  number
0  От 1 года до 3 лет   26152
1       От 3 до 6 лет   14511
2           Нет опыта    7197
3         Более 6 лет    1337


Детальный анализ вакансий показывает, что распределение вакансий по регионам напрямую зависит от размера региона:
на первом и втором месте оказались крупнейшие города России - Моква и Санкт-Петербург. На третьем месте оказался Минск. 
Скорее всего это определилось тем, что в Минске есть крупный IT кластер, а большинство вакансий в данной базе 
(см. исследование ниже) относится к IT.
По графику работы более половины вакансий предусматривают традиционный график работы:  постоянную работу с полной занятостью. Но на втором месте по численности оказалась удаленная работа при полной занятости, что говорит об изменении условий труда в связи с развитием интернета, глобализацией. Свою лепту в развитие удаленной работы внес Ковид.
Неожиданный результат дало распределение вакансий в зависимости от опыта работы: наибольшим спросом пользуются специалисты с опытом работы от года до трех лет (26152 вакансии), на втором месте специалисты с опытом работы от 3 до 6 лет. Возможно, это опять связано с тем, что большинство вакансий в базе так или иначе связано с IT и говорит о бурном развитии данной сферы: возникновение новых направлений, новых бизнесов. Возможно, что распределение вакансий связано с зарплатными возможностями: чем больше опыт работы, тем выше запрос соискателя по зарплате. Наименьшим спросом пользуются специалисты с опытом работы более 6 лет. Для ответа на вопрос почему это так, требуются дополнительные исследования. Предположение: вакансии с таким опытом работы подразумевают руководящие должности и более высокие зарплаты.
По зарплатам: к сожалению, возможная зарплата указана менее, чем у половины вакансий. Средние значения зарплатной вилки не дают возможности для анализа. Здесь больше информации для анализа дадут максимум и минимум, медианное значение зарплаты, а также распределение зарплат по регионам, отраслям, опыту работы и графику.

Задание 5. Анализ работодателей

In [9]:
# Распределение вакансий по работодателям
query = '''select em.name, count(v.id) number from employers em, vacancies v
where v.employer_id=em.id group by em.name order by number desc limit 20'''

vacancies_by_employers = pd.read_sql_query(query, connection)
print(vacancies_by_employers)

                        name  number
0                     Яндекс    1933
1                 Ростелеком     491
2                   Тинькофф     444
3                       СБЕР     428
4              Газпром нефть     331
5                ИК СИБИНТЕК     327
6                        МТС     292
7                    DataArt     247
8      Совкомбанк Технологии     204
9                 Первый Бит     176
10                      Ozon     175
11                       IBS     175
12                  Andersen     155
13                    Тензор     155
14                Спецремонт     151
15                        VK     150
16    Поляков Денис Иванович     127
17  Иннотех, Группа компаний     122
18           Bell Integrator     121
19                     ANCOR     118


In [10]:
# Распределение работодателей и вакансий по регионам
query = '''select ar.name, count(distinct em.id), count(v.id)  from areas ar
left join employers em on ar.id=em.area
left join vacancies v on v.area_id=ar.id
where v.id is null
group by ar.name
order by count(distinct em.id) desc'''

empl_vac_by_area = pd.read_sql_query(query, connection)
print(empl_vac_by_area)


                               name  count  count
0                            Россия    410      0
1                         Казахстан    207      0
2                Московская область     75      0
3                Краснодарский край     19      0
4                          Беларусь     18      0
..                              ...    ...    ...
588                        Кувандык      0      0
589                        Куйбышев      0      0
590  Куйбышево (Ростовская область)      0      0
591                        Култаево      0      0
592                        Кульсары      0      0

[593 rows x 3 columns]


In [11]:
# Количество вакансий, которые работодатели публикуют по разным регионам
query = '''select em.name employer, count(distinct v.area_id) areas from vacancies v
left join employers em on v.employer_id=em.id
group by em.name order by areas desc'''

empl_areas = pd.read_sql_query(query, connection)
print(empl_areas)

                     employer  areas
0                      Яндекс    181
1                  Ростелеком    152
2                  Спецремонт    116
3      Поляков Денис Иванович     88
4                   ООО ЕФИН      71
...                       ...    ...
14761                  UniSol      1
14762            UNISTORY LLC      1
14763                   UNIT6      1
14764     United Distribution      1
14765              UNITEL LLC      1

[14766 rows x 2 columns]


In [12]:
# Подсчет числа работодателей, у которых не указана сфера деятельности
query = '''select count(em.id) total  from employers em
        left join employers_industries ei on em.id=ei.employer_id
        group by ei.industry_id
        having ei.industry_id is null'''

industry_null = pd.read_sql_query(query, connection)

print('Число работодателей, у которых не указана сфера деятельности:', industry_null['total'][0])

Число работодателей, у которых не указана сфера деятельности: 8419


In [13]:
# Название третьего по алфавиту работодателя, который работает в 4 сферах детельности
query = '''select em.name employer from employers em
left join employers_industries ei on em.id=ei.employer_id
group by em.name
having count(ei.industry_id)=4 order by employer
offset 2 limit 1'''

employer_4 = pd. read_sql_query(query, connection)

print('Работодатель:', employer_4['employer'][0])

Работодатель: 2ГИС


In [14]:
# Число работодателей работающих в сфере разработки ПО
query = '''select ind.name industry, count(ei.employer_id) employers_cnt from employers_industries ei
left join industries ind on ei.industry_id=ind.id
group by ind.name
having ind.name='Разработка программного обеспечения'
'''

employer = pd.read_sql_query(query, connection)

print(employer.iloc[0])

industry         Разработка программного обеспечения
employers_cnt                                   3553
Name: 0, dtype: object


In [15]:
# Число вакансий Яндекса в городах-миллионниках

query = '''(select ar.name, count(v.id) cnt from vacancies v
left join employers em on v.employer_id=em.id
left join areas ar on v.area_id=ar.id
where ar.name in ('Москва','Санкт-Петербург','Новосибирск','Екатеринбург','Казань','Нижний Новгород','Челябинск','Красноярск','Самара','Уфа','Ростов-на-Дону','Омск','Краснодар','Воронеж','Пермь','Волгоград')
and em.name='Яндекс'
group by ar.name)
union all
(select 'total',count(v.id) cnt from vacancies v
left join employers em on v.employer_id=em.id
left join areas ar on v.area_id=ar.id
where ar.name in ('Москва','Санкт-Петербург','Новосибирск','Екатеринбург','Казань','Нижний Новгород','Челябинск','Красноярск','Самара','Уфа','Ростов-на-Дону','Омск','Краснодар','Воронеж','Пермь','Волгоград')
and em.name='Яндекс'
) order by cnt'''

yandex_region_vacancies = pd.read_sql_query(query, connection)
print(yandex_region_vacancies)

               name  cnt
0              Омск   21
1         Челябинск   22
2        Красноярск   23
3         Волгоград   24
4             Пермь   25
5            Казань   25
6    Ростов-на-Дону   25
7               Уфа   26
8            Самара   26
9         Краснодар   30
10          Воронеж   32
11      Новосибирск   35
12  Нижний Новгород   36
13     Екатеринбург   39
14  Санкт-Петербург   42
15           Москва   54
16            total  485


Наибольшее число работодателей в данной базе являются российскими (410 компаний) (что неудивительно, так как изучаемая база данных принадлежит российскому агенству). На втором месте находится Казахстан (207 компаний), что говорит о тесном экономическом сотрудничестве наших стран. Если посмотреть распределение вакансий по регионам, то суммарное число вакансий по двум крупнейшим городам Казахстана (Алма-Аты и Нур-Султан) составляет 2732, что почти догоняет число вакансий в Санкт-Петербурге (2851).
Из 23501 работодателя в БД у более, чем трети (8419) не указана сфера деятельности (возможно, ее можем определить по характеру вакансий), у некоторых указано более одной сферы деятельности. Здесь возможны варианты, как компания работает в нескольких направлениях одной сферы (например, разработка ПО, сопровождение ПО), так и холдинг, работающий в разных направлениях (например Яндекс как разработчик ПО, Яндекс-браузер, Яндекс-доставка). Требуются дополнительные исследования.
Крупнейшими работодателями в данной базе являются крупные межрегиональные российские холдинги, большинство из которых работают в сфере высоких технологий: Яндекс, Ростелеком, СБЕР, Тинькофф, Газпром, IBS, VK и т. д. На первом месте по всем параметрам располагается компания Яндекс: 1933 вакансии в 181 регионе. Это опять приводит нас к тому, что больше всег вакансий относятся к сфере IT. РАспределение вакансий Яндекса по городам не дает нам ничего нового: чем крупнее город, тем больше в нем вакансий. Первое и второе места: Москва и Санкт-Петербург.

Задание 6. Анализ вакансий в области DataScience

In [16]:
# Число вакансий, имеющих отношение к данным

query = '''select count(name) from vacancies where lower(name) like '%data%' or lower(name) like '%данн%'
        '''
vacancies_total = pd.read_sql_query(query, connection)

print('Число вакансий, имеющих отношение к данным:', vacancies_total['count'][0])

Число вакансий, имеющих отношение к данным: 1771


In [17]:
# дополнительный запрос для выборки вакансий Data Science
ds_vacancies = '''with ds_vacancies as 
                (select * from vacancies 
                where lower(name) like '%data scientist%' or lower(name) like '%data science%' 
                or lower(name) like '%исследователь данных%' or (name like '%ML%' and name not like '%HTML%') 
                or lower(name) like '%machine learning%' or lower(name) like '%машинн%обучен%') 
                '''

In [18]:
# Вакансии Data Science для Junior
query = ds_vacancies + '''select count(*) from ds_vacancies 
        where lower(name) like '%junior%' or employment='Стажировка' or experience='Нет опыта'
        '''

junior_ds_vac = pd.read_sql_query(query, connection)
print('Число вакансий для начинающих:',junior_ds_vac['count'][0])

Число вакансий для начинающих: 51


In [19]:
# Число вакансий DS с ключевым навыком "SQL" или "postreg"
query = ds_vacancies + '''select count(*) from ds_vacancies 
        where key_skills like '%SQL%' or lower(key_skills) like '%postgres%'
        '''

sql_skill_vac = pd.read_sql_query(query, connection)

print('Число вакансий со знанием SQL:', sql_skill_vac['count'][0])

Число вакансий со знанием SQL: 201


In [20]:
# Число вакансий DS с ключевым навыком Python
query = ds_vacancies + '''select count(*) from ds_vacancies where key_skills like '%Python%'
        '''

python_skill_vac = pd.read_sql_query(query, connection)

print('Число вакансий со знанием Python:', python_skill_vac['count'][0])

Число вакансий со знанием Python: 351


In [21]:
# Среднее число навыков, указываемых в вакансиях DS
query = ds_vacancies + '''select avg(array_length(regexp_split_to_array(key_skills,'\t'),1)) from ds_vacancies'''

key_skills_avg = pd.read_sql_query(query, connection)
print(key_skills_avg)

        avg
0  6.406032


In [22]:
# Средняя зарплата в зависимости от опыта работы
query = ds_vacancies + ''' select experience, avg(coalesce((salary_from+salary_to)/2,salary_from,salary_to)) salary
from ds_vacancies where salary_from is not null or salary_to is not null
group by experience'''

#print(ds_vacancies, query)

salary_avg_by_exp = pd.read_sql_query(query, connection)

print(salary_avg_by_exp)

           experience         salary
0       От 3 до 6 лет  243114.666667
1  От 1 года до 3 лет  139674.750000
2           Нет опыта   74642.857143


7. Дополнительные исследования

In [23]:
# Распределение средних зарплат в зависимости от опыта работы
query = '''select experience, round(avg(coalesce((salary_from+salary_to)/2,salary_from,salary_to))) salary
from vacancies where salary_from is not null or salary_to is not null
group by experience order by salary desc'''

salary_avg_by_exp = pd.read_sql_query(query, connection)
print(salary_avg_by_exp)

           experience    salary
0         Более 6 лет  166256.0
1       От 3 до 6 лет  133211.0
2  От 1 года до 3 лет   76541.0
3           Нет опыта   40472.0


In [24]:
# Дополнительные запросы для выборок из БД

# Вакансии для разработчиков ПО
programming_vacancies = '''programming_vacancies as
(select * from vacancies 
where lower(name) like '%developer%' or lower(name) like '%development%' or lower(name) like '%разработчик%' or 
lower(name) like '%разработк%' or lower(name) like '%программист%'
except
select * from vacancies
where lower(name) like '%database%' or name like '% DB%' or lower(name) like '%баз%данных%' 
or name like '%БД%' or lower(name) like '%business development%' 
or lower(name) like '%data%' or lower(name) like '%данн%')
'''

# Вакансии для специалистов в других областях IT (администрирование, поддержка, итп)
it_spec_vacancies = '''it_spec_vacancies as
(select * from vacancies 
where lower(name) like '%support%' or lower(name) like '%customer service%' or
lower(name) like '%help%desk%' or lower(name) like '%тех%поддержк%' or 
lower(name) like '%system administrator%' or lower(name) like '%системный администратор%' or
lower(name) like 'it specialist' or lower(name) like 'it engineer' or lower(name) like '%it project manager%' or
lower(name) like 'it специалист' or lower(name) like 'it инженер' or lower(name) like '%менеджер it%проекта%')
'''

# Вакансии для специалистов по базам данных
# db_vacancies = '''db_vacancies as
# (select * from vacancies 
# where lower(name) like '%database%' or name like '% DB%' or lower(name) like '%баз%данных%' or name like '%БД%')
# '''

# Вакансии для специалистов в области обработки данных
data_vacancies = '''data_vacancies as
(select * from vacancies where lower(name) like '%data%' or lower(name) like '%данн%')'''
# except
# select * from vacancies 
# where lower(name) like '%database%' or name like '% DB%' or lower(name) like '%баз%данных%' or name like '%БД%')'''

In [25]:
# Статистика вакансий разработчиков
query = 'with '+ programming_vacancies + ''' select 'vacancies' stat, count(*) development
from programming_vacancies
union
select 'salary_max', max(salary_to) from programming_vacancies where salary_to is not null
union
select 'salary_min', min(salary_from) from programming_vacancies where salary_from is not null and salary_from >=1000
union
select 'salary_avg', round(avg(coalesce((salary_from+salary_to)/2,salary_from,salary_to))) salary_avg
from programming_vacancies where salary_from is not null or salary_to is not null
order by stat'''

salary_result_df = pd.read_sql_query(query, connection)
#print(salary_result_df)

# Статистика вакансий IT специалистов
query = 'with '+ it_spec_vacancies + ''' select 'vacancies' stat, count(*) it_specialists
from it_spec_vacancies
union
select 'salary_max', max(salary_to) from it_spec_vacancies where salary_to is not null
union
select 'salary_min', min(salary_from) from it_spec_vacancies where salary_from is not null and salary_from >=1000
union
select 'salary_avg', round(avg(coalesce((salary_from+salary_to)/2,salary_from,salary_to))) salary_avg
from it_spec_vacancies where salary_from is not null or salary_to is not null
order by stat'''

salary_result_df['it_specialists'] = pd.read_sql_query(query, connection)['it_specialists']

# Статистика вакансий специалистов в области БД
#query = 'with '+ db_vacancies + ''' select 'vacancies' stat, count(*) db_specialists
#from db_vacancies
#union
#select 'salary_max', max(salary_to) from db_vacancies where salary_to is not null
#union
#select 'salary_min', min(salary_from) from db_vacancies where salary_from is not null and salary_from >=1000
#union
#select 'salary_avg', round(avg(coalesce((salary_from+salary_to)/2,salary_from,salary_to))) salary_avg
#from db_vacancies where salary_from is not null or salary_to is not null
#order by stat'''

#salary_result_df['db_specialists'] = pd.read_sql_query(query, connection)['db_specialists']

# Статистика вакансий в области обработки данных
query = 'with '+ data_vacancies+ ''' select 'vacancies' stat, count(*) data_specialists
from data_vacancies
union
select 'salary_max', max(salary_to) from data_vacancies where salary_to is not null
union
select 'salary_min', min(salary_from) from data_vacancies where salary_from is not null and salary_from >=1000
union
select 'salary_avg', round(avg(coalesce((salary_from+salary_to)/2,salary_from,salary_to))) salary_avg
from data_vacancies where salary_from is not null or salary_to is not null
order by stat
'''

salary_result_df['data_specialists'] = pd.read_sql_query(query, connection)['data_specialists']

# Статистика вакансий по всей базе данных
query = ''' select 'vacancies' stat, count(*) total_vacancies from vacancies
union
select 'salary_max', max(salary_to) from vacancies where salary_to is not null
union
select 'salary_min', min(salary_from) from vacancies where salary_from is not null and salary_from >=1000
union
select 'salary_avg', round(avg(coalesce((salary_from+salary_to)/2,salary_from,salary_to))) salary_avg
from vacancies where salary_from is not null or salary_to is not null
order by stat
'''

salary_result_df['total_vacancies'] = pd.read_sql_query(query, connection)['total_vacancies']

print(salary_result_df)

         stat  development  it_specialists  data_specialists  total_vacancies
0  salary_avg     118897.0         52315.0          116919.0          84315.0
1  salary_max    1000000.0        350000.0          700000.0        1000000.0
2  salary_min       1000.0          2922.0           13835.0           1000.0
3   vacancies      15912.0          5134.0            1771.0          49197.0


#### Закрытие коннекта

In [26]:
connection.close()